# Cheat Sheet

In [1]:
import jijmodeling as jm

## 総和

### 決定変数の総和

#### Plain API

In [2]:
problem = jm.Problem("BasicSum")
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
problem += x.sum()

problem

Problem(name="BasicSum", sense=MINIMIZE, objective=sum(x), constraints=[])

#### Decorator API

In [3]:
@jm.Problem.define("BasicSum")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    x = problem.BinaryVar(shape=(N,))
    problem += x.sum()

problem

Problem(name="BasicSum", sense=MINIMIZE, objective=sum(x), constraints=[])

### 係数付き決定変数の総和

#### Plain API

In [4]:
problem = jm.Problem("WeightedSum")
a = problem.Float("a", ndim=1)
N = problem.DependentVar("N", a.len_at(0))
x = problem.BinaryVar("x", shape=(N,))
problem += jm.sum(a * x)

problem

Problem(name="WeightedSum", sense=MINIMIZE, objective=sum(a * x), constraints=[])

#### Decorator API

In [5]:
@jm.Problem.define("WeightedSum")
def problem(problem: jm.DecoratedProblem):
    a = problem.Float(ndim=1)
    N = problem.DependentVar(a.len_at(0))
    x = problem.BinaryVar(shape=(N,))
    problem += (a * x).sum()

problem

Problem(name="WeightedSum", sense=MINIMIZE, objective=sum(a * x), constraints=[])

### 添字集合に沿った決定変数の総和

#### Plain API

In [6]:
problem = jm.Problem("SumAlongSet")
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
C = problem.Natural("C", ndim=1)
problem += jm.sum(jm.map(lambda i: x[i], C))

problem

Problem(name="SumAlongSet", sense=MINIMIZE, objective=sum(C.map(lambda (i: natural): x[i])), constraints=[])

#### Decorator API

In [7]:
@jm.Problem.define("SumAlongSet")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    x = problem.BinaryVar(shape=(N,))
    C = problem.Natural(ndim=1)
    problem += jm.sum(x[i] for i in C)

problem

Problem(name="SumAlongSet", sense=MINIMIZE, objective=sum(C.map(lambda (i: natural): x[i])), constraints=[])

### 辺集合に沿った決定変数の総和

#### Plain API

In [8]:
problem = jm.Problem("SumAlongEdgeSet")
V = problem.Natural("V")
E = problem.Graph("E")
x = problem.BinaryVar("x", shape=(V,))
problem += jm.map(lambda i, j: x[i] * x[j], E).sum()

problem

Problem(name="SumAlongEdgeSet", sense=MINIMIZE, objective=sum(E.map(lambda ((i, j): Tuple[natural, natural]): x[i] * x[j])), constraints=[])

#### Decorator API

In [9]:
@jm.Problem.define("SumAlongEdgeSet")
def problem(problem: jm.DecoratedProblem):
    V = problem.Natural()
    E = problem.Graph()
    x = problem.BinaryVar(shape=(V,))
    problem += jm.sum(x[i] * x[j] for (i, j) in E)

problem

Problem(name="SumAlongEdgeSet", sense=MINIMIZE, objective=sum(E.map(lambda ((i, j): Tuple[natural, natural]): x[i] * x[j])), constraints=[])

### 条件付きの総和

#### Plain API

In [10]:
problem = jm.Problem("ConditionalSum")
N = problem.Natural("N")
J = problem.Float("J", shape=(N, N))
x = problem.BinaryVar("x", shape=(N,))
problem += jm.map(
    lambda i: jm.filter(lambda j: i > j, N).map(lambda j: J[i, j] * x[i] * x[j]).sum(),
    N
).sum()

problem

Problem(name="ConditionalSum", sense=MINIMIZE, objective=sum(N.map(lambda (i: natural): sum(N.filter(lambda j: i > j).map(lambda (j: natural): J[i, j] * x[i] * x[j])))), constraints=[])

#### Decorator API

In [11]:
@jm.Problem.define("ConditionalSum")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    J = problem.Float(shape=(N, N))
    x = problem.BinaryVar(shape=(N,))
    problem += jm.sum(J[i, j] * x[i] * x[j] for i in N for j in N if i > j)

problem

Problem(name="ConditionalSum", sense=MINIMIZE, objective=sum(N.flat_map(lambda (i: natural): N.map(lambda (j: natural): (i, j))).filter(lambda (i, j): i > j).map(lambda ((i, j): Tuple[natural, natural]): J[i, j] * x[i] * x[j])), constraints=[])

### 行列の対角要素を除く総和

#### Plain API

In [12]:
problem = jm.Problem("NonDiagonalSum")
N = problem.Natural("N")
J = problem.Float("J", shape=(N, N))
problem += jm.map(
    lambda i: jm.filter(lambda j: i != j, N).map(lambda j: J[i, j]).sum(),
    N
).sum()

problem

Problem(name="NonDiagonalSum", sense=MINIMIZE, objective=sum(N.map(lambda (i: natural): sum(N.filter(lambda j: i != j).map(lambda (j: natural): J[i, j])))), constraints=[])

#### Decorator API

In [13]:
@jm.Problem.define("NonDiagonalSum")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    J = problem.Float(shape=(N, N)) 
    problem += jm.sum(J[i, j] for i in N for j in N if i != j)

problem

Problem(name="NonDiagonalSum", sense=MINIMIZE, objective=sum(N.flat_map(lambda (i: natural): N.map(lambda (j: natural): (i, j))).filter(lambda (i, j): i != j).map(lambda ((i, j): Tuple[natural, natural]): J[i, j])), constraints=[])

### 別のインデックスに依存した総和

#### Plain API

In [14]:
problem = jm.Problem("DependentSum")
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
a = problem.Natural("a", ndim=1)
M = problem.DependentVar("M", a.len_at(0))
problem += jm.sum(jm.flat_map(lambda i: a[i].map(lambda j: x[j]), M))

problem

Problem(name="DependentSum", sense=MINIMIZE, objective=sum(M.flat_map(lambda (i: natural): a[i].map(lambda (j: natural): x[j]))), constraints=[])

#### Decorator API

In [15]:
@jm.Problem.define("DependentSum")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    x = problem.BinaryVar(shape=(N,))
    a = problem.Natural(ndim=1)
    M = problem.DependentVar(a.len_at(0))
    problem += jm.sum(x[j] for i in M for j in a[i])

problem

Problem(name="DependentSum", sense=MINIMIZE, objective=sum(M.flat_map(lambda (i: natural): a[i].map(lambda (j: natural): (i, j))).map(lambda ((i, j): Tuple[natural, natural]): x[j])), constraints=[])

## 制約条件
### One-hot 制約

#### Plain API

In [16]:
problem = jm.Problem("OneHot")
N = problem.Natural("N")
x = problem.BinaryVar("x", shape=(N,))
problem += problem.Constraint("onehot", x.sum() == 1)

problem

Problem(name="OneHot", sense=MINIMIZE, objective=0, constraints={onehot: [Constraint(name="onehot", sense=EQUAL, left=sum(x), right=1, shape=Scalar(Binary)),],})

#### Decorator API

In [17]:
@jm.Problem.define("OneHot")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    x = problem.BinaryVar(shape=(N,))
    problem += problem.Constraint("onehot", x.sum() == 1)

problem

Problem(name="OneHot", sense=MINIMIZE, objective=0, constraints={onehot: [Constraint(name="onehot", sense=EQUAL, left=sum(x), right=1, shape=Scalar(Binary)),],})

### K-hot 制約

#### Plain API

In [18]:
problem = jm.Problem("KHot")
N = problem.Natural("N")
K = problem.Natural("K")
x = problem.BinaryVar("x", shape=(N,))
problem += problem.Constraint("k_hot", x.sum() == K)

problem

Problem(name="KHot", sense=MINIMIZE, objective=0, constraints={k_hot: [Constraint(name="k_hot", sense=EQUAL, left=sum(x), right=K, shape=Scalar(Natural)),],})

#### Decorator API

In [19]:
@jm.Problem.define("KHot")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    K = problem.Natural()
    x = problem.BinaryVar(shape=(N,))
    problem += problem.Constraint("k_hot", x.sum() == K)

problem

Problem(name="KHot", sense=MINIMIZE, objective=0, constraints={k_hot: [Constraint(name="k_hot", sense=EQUAL, left=sum(x), right=K, shape=Scalar(Natural)),],})

### 2 次元バイナリ変数の各列に対する K-hot 制約

#### Plain API

In [20]:
problem = jm.Problem("2D K-Hot")
K = problem.Natural("K", ndim=1)
N = problem.DependentVar("N", K.len_at(0))
M = problem.Natural("M")
x = problem.BinaryVar("x", shape=(N, M))
problem += problem.Constraint("2d k-hot", x.sum(axis=1) == K)

problem

Problem(name="2D K-Hot", sense=MINIMIZE, objective=0, constraints={2d k-hot: [Constraint(name="2d k-hot", sense=EQUAL, left=x.sum(1), right=K, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: None, subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/3312631698.py", start: SrcLoc { line: 5, column: 4 }, end: SrcLoc { line: 5, column: 40 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/3312631698.py", start: SrcLoc { line: 5, column: 4 }, end: SrcLoc { line: 5, column: 40 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Natural }, src_span: NoSrcSpan } }), scalar: Natural }),],})

#### Decorator API

In [21]:
@jm.Problem.define("2D K-Hot")
def problem(problem: jm.DecoratedProblem):
    K = problem.Natural(ndim=1)
    N = problem.DependentVar(K.len_at(0))
    M = problem.Natural()
    x = problem.BinaryVar(shape=(N, M))
    problem += problem.Constraint("2d k-hot", x.sum(axis=1) == K)

problem

Problem(name="2D K-Hot", sense=MINIMIZE, objective=0, constraints={2d k-hot: [Constraint(name="2d k-hot", sense=EQUAL, left=x.sum(1), right=K, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: None, subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/313414419.py", start: SrcLoc { line: 6, column: 8 }, end: SrcLoc { line: 6, column: 39 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/313414419.py", start: SrcLoc { line: 6, column: 8 }, end: SrcLoc { line: 6, column: 39 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Natural }, src_span: NoSrcSpan } }), scalar: Natural }),],})

### 各集合に対する K-hot 制約

#### Plain API

In [22]:
problem = jm.Problem("KHotOverSet")
N = problem.Natural("N")
C = problem.Natural("C", jagged=True, ndim=2)
M = problem.DependentVar("M", C.len_at(0))
K = problem.Natural("K", shape=(M,))
x = problem.BinaryVar("x", shape=(N,))
problem += problem.Constraint(
    "k-hot_constraint", lambda a: C[a].map(lambda i: x[i]).sum() == K[a], domain=M
)

problem

Problem(name="KHotOverSet", sense=MINIMIZE, objective=0, constraints={k-hot_constraint: [Constraint(name="k-hot_constraint", , lambda a: sum(C[a].map(lambda (i: natural): x[i])) == K[a], domain=set(M)),],})

#### Decorator API

In [23]:
@jm.Problem.define("KHotOverSet")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    C = problem.Natural(jagged=True, ndim=2)
    M = problem.DependentVar(C.len_at(0))
    K = problem.Natural(shape=(M,))
    x = problem.BinaryVar(shape=(N,))
    problem += problem.Constraint(
        "k-hot_constraint", (jm.sum(x[i] for i in C[a]) == K[a] for a in M),
    )

problem

Problem(name="KHotOverSet", sense=MINIMIZE, objective=0, constraints={k-hot_constraint: [Constraint(name="k-hot_constraint", , lambda a: sum(C[a].map(lambda (i: natural): x[i])) == K[a], domain=set(M)),],})

### 線形不等式制約

#### Plain API

In [24]:
problem = jm.Problem("LinearInequality")
w = problem.Float("w", ndim=1)
N = problem.DependentVar("N", w.len_at(0))
W = problem.Float("W")
x = problem.BinaryVar("x", shape=(N,))
problem += problem.Constraint("weight", (w * x).sum() <= W)

problem

Problem(name="LinearInequality", sense=MINIMIZE, objective=0, constraints={weight: [Constraint(name="weight", sense=LESS_THAN_EQUAL, left=sum(w * x), right=W, shape=Scalar(Float)),],})

#### Decorator API

In [25]:
@jm.Problem.define("LinearInequality")
def problem(problem: jm.DecoratedProblem):
    w = problem.Float(ndim=1)
    N = problem.DependentVar(w.len_at(0))
    W = problem.Float()
    x = problem.BinaryVar(shape=(N,))
    problem += problem.Constraint("weight", (w * x).sum() <= W)

problem

Problem(name="LinearInequality", sense=MINIMIZE, objective=0, constraints={weight: [Constraint(name="weight", sense=LESS_THAN_EQUAL, left=sum(w * x), right=W, shape=Scalar(Float)),],})

### SOS1 不等式制約

#### Plain API

In [26]:
problem = jm.Problem("SOS-1")
N = problem.Natural("N")
M = problem.Float("M", shape=(N,))
a = problem.ContinuousVar("a", shape=N, lower_bound=0, upper_bound=M)
x = problem.BinaryVar("x", shape=N)
problem += problem.Constraint("SOS1", x.sum() <= 1)
problem += problem.Constraint("Big-M", a <= M * x)

problem

Problem(name="SOS-1", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", sense=LESS_THAN_EQUAL, left=a, right=M * x, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: Some(SetStyle { element: None, kind: Natural }), subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/1199227916.py", start: SrcLoc { line: 2, column: 4 }, end: SrcLoc { line: 2, column: 24 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/1199227916.py", start: SrcLoc { line: 2, column: 4 }, end: SrcLoc { line: 2, column: 24 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Float }, src_span: NoSrcSpan } }), scalar: Float }),],SOS1: 

#### Decorator API

In [27]:
@jm.Problem.define("SOS-1")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    M = problem.Float(shape=(N,))
    a = problem.ContinuousVar(shape=N, lower_bound=0, upper_bound=M)
    x = problem.BinaryVar(shape=N)
    problem += problem.Constraint("SOS1", x.sum() <= 1)
    problem += problem.Constraint("Big-M", a <= M * x)

problem

Problem(name="SOS-1", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", sense=LESS_THAN_EQUAL, left=a, right=M * x, shape=SameTensorLike { spec: Tensor(TensorSpec { shape: [Specified(Located { inner: RangeNat { upper_bound: Located { inner: FVar { name: "N", ext: WithType { type_: Scalar { mode: Static, kind: Natural }, inner: LaTeXMetadata { custom_latex: None, set_style: Some(SetStyle { element: None, kind: Natural }), subscript_styles: [] } } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/3596302424.py", start: SrcLoc { line: 3, column: 8 }, end: SrcLoc { line: 3, column: 25 } }) } }, src_span: Real(RealSrcSpan { filename: "/var/folders/94/q3gqrl392hs5234_yg7rxpgm0000gn/T/ipykernel_35643/3596302424.py", start: SrcLoc { line: 3, column: 8 }, end: SrcLoc { line: 3, column: 25 } }) })], element: Located { inner: Scalar { mode: Dynamic, kind: Float }, src_span: NoSrcSpan } }), scalar: Float }),],SOS1: 

### Big-M 不等式制約

#### Plain API

In [28]:
problem = jm.Problem("BigM")
N = problem.Natural("N")
c = problem.Float("c", shape=(N, N))
M = problem.Float("M")
x = problem.BinaryVar("x", shape=(N, N))
e = problem.Float("e", shape=(N,))
l = problem.Float("l", shape=(N,))
t = problem.IntegerVar("t", shape=(N,), lower_bound=e, upper_bound=l)
non_diagonals = jm.product(N, N).filter(lambda i, j: i != j)
problem += problem.Constraint(
    "Big-M",
    lambda i, j: t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j],
    domain=non_diagonals,
)

problem

Problem(name="BigM", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", , lambda (i, j): t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j], domain=set((N, N)).filter(lambda (i, j): i != j)),],})

#### Decorator API

In [29]:
@jm.Problem.define("BigM")
def problem(problem: jm.DecoratedProblem):
    N = problem.Natural()
    c = problem.Float(shape=(N, N))
    M = problem.Float()
    x = problem.BinaryVar(shape=(N, N))
    e = problem.Float(shape=(N,))
    l = problem.Float(shape=(N,))
    t = problem.IntegerVar(shape=(N,), lower_bound=e, upper_bound=l)
    problem += problem.Constraint(
        "Big-M",
        (
            t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j]
            for i in N
            for j in N
            if i != j
        ),
    )

problem

Problem(name="BigM", sense=MINIMIZE, objective=0, constraints={Big-M: [Constraint(name="Big-M", , lambda (i, j): t[i] + c[i, j] - M * (1 - x[i, j]) <= t[j], domain=N.flat_map(lambda (i: natural): N.map(lambda (j: natural): (i, j))).filter(lambda (i, j): i != j)),],})